# Fs-Mixture-of-Gaussian--Gabasova

Author: Evelina Gabasova  
Transcribed to a Jupyter notebook by Ian Buckley.

## Overview

This notebook, which can be conveniently [hosted on Azure](https://notebooks.azure.com/), is a transcription of the [F#](http://fsharp.org/) program [`mixture-of-gaussians`](https://github.com/evelinag/probabilistic-programming/blob/master/code/mixture_of_gaussians.fsx) developed by Evelina Gabasova as part of her presentation [How to look like a statistician: a developer's guide to probabilistic programming](https://channel9.msdn.com/Events/FSharp-Events/fsharpConf-2018/06), part of [FSharpConf](http://fsharpconf.com/) on 6 April 2018. This particular example models the responses to a salary survey, making use of a common design pattern in functional programming, useful for expressing workflows, called the [**computation expression**](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/computation-expressions) (also known as in math & computer science circles & by the Haskell & Scala communities as the [**monad**](https://en.wikipedia.org/wiki/Monad_(functional_programming)). 

##### Notebook extensions

Because this is a long notebook, & a mixture of valuable code & less valuable experiments, it is a really good idea to turn on a couple of notebook extensions **Edit > nbextensions config** to open a new browser tab, & then select the following extensions:
* Collapsible headings
* Intitialization cells (allows you to conveniently run only specific cells   

Having selected those check boxes, reload the tab containing this notebook for the changes to take effect.

## `Load` & `open` .net packages using `Paket`

### Steps when running on Azure

#### Run once in a given Azure library to create script

In [3]:
#load "Paket.fsx"

When using Azure, run the following to generate the file `Paket.Generated.Refs.fsx`, to be `load`ed each time that the notebook is run with a fresh kernel. The file is put in either 
`/home/nbuser/IfSharp/bin` or 
`/home/nbuser/library`

The following installs the selected packages. It may take a minute to run on first use with a particular kernel.

In [4]:
Paket.Package
  [ "MathNet.Numerics"
    "MathNet.Numerics.FSharp"
    "FSharp.Data"
  ]

#### Run always for restarted kernel 

 `Load` the file `Paket.Generated.Refs.fsx` each time that the notebook is run with a fresh kernel when on Azure.

In [5]:
#load "Paket.Generated.Refs.fsx"

### Steps for local execution

In [21]:
// On Azure, use the (Unix) terminal window to find these files in ~/IfSharp/bin/
#r "packages/MathNet.Numerics/lib/net40/MathNet.Numerics.dll"
#r "packages/FSharp.Data/lib/net45/FSharp.Data.dll"

### Common steps for Azure or local execution

In [6]:
open System
open MathNet.Numerics
open FSharp.Data

## Define types, computation expressions, functions, models

### Define types & helper functions for outcomes, distributions

#### Utilities

In [7]:
let binSize = 1000.0

In [8]:
let roundToBin x =
  let m = x % binSize
  if m >= binSize/2.0 then // round up
    x + binSize - m
  else // round down
    x - m

#### `DistributionValue`

Whereas the Monty Hall problem has discrete outcomes (Car or Goat), this problem requires  distributions with continuous random variables taking values on the real line.

In [9]:
type DistributionValue = float

#### `Distribution`

The parameters for the distributions are `float`s

In [10]:
type Distribution = 
  | Gaussian of mean:float * variance:float
  | Bernoulli of probability:float

In [11]:
let salaries = Gaussian(1000.0, 200.0)
let mistake = Bernoulli(0.1)

#### `ProbabilisticComputation`

A `ProbabilisticComputation` is a (generic) union type consisting of either:
* `Sample` is a tuple of 
    * a `Distribution` (described by its parameters) and 
    * a function that maps an input of type `DistributionValue` (alias for `float`) to a `ProbabilisticComputation`
* `Return` is just an alias for the provided wrapped type `'T`

In [12]:
type ProbabilisticComputation<'T> = 
  | Sample of Distribution * (DistributionValue -> ProbabilisticComputation<'T>)
  | Return of 'T   

#### `ProbabilisticComputationBuilder`

The computation expression builder must implement the `Return` and `Bind` methods.
* **`Return`** Lifts a single value into the elevated world `a -> E<a>`
* **`Bind`**  Allows you to compose world-crossing (“monadic”) functions `(a->E<b>) -> E<a> -> E<b>`

In [13]:
type ProbabilisticComputationBuilder() = 
  member x.Return(v) = Return v
  member x.Bind(dist, f) = Sample(dist, f) 

In [14]:
let prob = ProbabilisticComputationBuilder()

#### Diagram of `bind` function from F# for fun & profit

Diagram of `bind` function from F# for fun & profit. See [bind](https://fsharpforfunandprofit.com/posts/elevated-world-2/#bind). In the computation expression, due to syntactic sugar, `bind` method appears as `let!`.
In this concrete example the correspondences are:
* the *elevated world* is a (generic) type `ProbabilisticComputation<DistributionValue>`, namely either a `Sample` or a `Return`.
* the type `a` is always an outcome of type `DistributionValue`
* ditto for type `b`??

![Image of bind function from F# for fun & profit](https://fsharpforfunandprofit.com/assets/img/vgfp_bind.png)

#### `model`

In [15]:
let model = 
  prob {
    let! yearlySalary = Gaussian(20000.0, 10000.0)
    let! mistake = Bernoulli(0.5)

    if mistake = 1.0 then 
      return yearlySalary / 12.0
    else 
      return yearlySalary
  }  

#### `varyParameters`

List of distributions with different parameters to explore

In [16]:
let varyParameters dist =
  match dist with
  | Gaussian(m, v) ->
     [ for mean in 0.0 .. 2000.0 .. 30000.0 do
        for var in 10000000.0 .. 1000000.0 .. 100000000.0 do 
          yield Gaussian(mean, var) ]
  | Bernoulli(_) ->
      [ for pt in 0.0 .. 0.05 .. 1.0 do
          yield Bernoulli(pt) ]

#### `discretize`

Discrete versions of the distributions - bin the support region of the distribution  
Returns discretized value and its probability of occuring (as a `list` of `tuple`s)

In [17]:
let discretize dist =
  match dist with 
  | Bernoulli(prob) -> 
      [1.0, prob; 0.0, (1.0 - prob) ]
  | Gaussian(mean, var) -> 
      [ for x in 0.0 .. binSize .. 110000.0 -> 
        // CDF (cummulative density function):   p(x <= N(mean,var)) 
        x, Distributions.Normal.CDF(mean, sqrt(var), x)]
      |> List.windowed 2
      |> List.map (fun w -> 
          fst w.[1], snd w.[1] - snd w.[0])

#### `enumerate`

Go through a distribution and obtain the probability for individual bins  
Uses the computational expression to go through the distributions in our model

In [18]:
let rec enumerate dists choices model = seq {
  match model with
  | Sample(dist, f) ->
     for dist in varyParameters dist do
       for v, p in discretize dist do 
         yield! enumerate (dist::dists) ((v,p)::choices) (f v)
  | Return v -> 
      yield List.rev dists, (v, List.rev choices) }

#### `histograms`

Compute probability for discrete bins based on sampled distribution

In [19]:
let histograms = 
  enumerate [] [] model
  |> Seq.map (fun (dist, (value, trace)) ->
      dist, (roundToBin value, trace))
  |> Seq.distinct
  |> Seq.map (fun (dist, (value, trace)) -> 
    let logp = 
      trace 
      |> List.map (fun (_, p) -> if p = 0.0 then 1e-10 else p)
      |> List.map log
      |> List.fold (+) 0.0
    dist, value, logp)
  |> Seq.groupBy (fun (dist, _, _) -> dist)
  |> Seq.map (fun (dist, values) ->
      dist, 
      values |> Seq.map (fun (_, v, p) -> v, p)
      |> Array.ofSeq
      |> Array.sortBy fst)
  |> Array.ofSeq  

Expression evaluation failed: Object reference not set to an instance of an object
NullReferenceExceptionObject reference not set to an instance of an object
  at Microsoft.FSharp.Core.Operators.FailurePattern (System.Exception error) [0x00001] in <5939249c904cf4daa74503839c243959>:0 
  at Microsoft.FSharp.Compiler.ErrorLogger+ErrorLoggerExtensions.ReraiseIfWatsonable (System.Exception exn) [0x0002d] in <590496d0ddab8ea7a7450383d0960459>:0 
  at Microsoft.FSharp.Compiler.ErrorLogger+ErrorLoggerExtensions.ErrorLogger.ErrorRecovery (Microsoft.FSharp.Compiler.ErrorLogger+ErrorLogger x, System.Exception exn, Microsoft.FSharp.Compiler.Range+range m) [0x000d2] in <590496d0ddab8ea7a7450383d0960459>:0 
  at Microsoft.FSharp.Compiler.ErrorLogger+ErrorLoggerExtensions.ErrorLogger.StopProcessingRecovery (Microsoft.FSharp.Compiler.ErrorLogger+ErrorLogger x, System.Exception exn, Microsoft.FSharp.Compiler.Range+range m) [0x000b5] in <590496d0ddab8ea7a7450383d0960459>:0 
  at Microsoft.FSharp.Compil

#### `compareDataHistogram`

Compute likelihood of the data given the simulated histogram

In [20]:
let compareDataHistogram (dist, values) data =
  let histLogProbability = 
    values
    |> Array.append [| (0.0, 0.0) |]
    |> Array.windowed 2
    |> Array.map (fun bin ->
        let p = bin.[1] |> snd
        p, 
        data 
        |> Array.filter (fun x -> 
            x > fst bin.[0] && x <= fst bin.[1])
        |> Array.length
        |> float)
    |> Array.fold (fun pAcc (logp, x) -> x * logp + pAcc) 0.0
  dist, histLogProbability

#### `mostLikelyDistribution`

In [21]:
let mostLikelyDistribution hists data =
  hists
  |> Array.map (fun distributionHist -> compareDataHistogram distributionHist data)
  |> Array.sortByDescending snd
  |> Array.take 10

### Apply to Stack Overflow data

Apply to Stack Overflow data

In [ ]:
let [<Literal>] file = __SOURCE_DIRECTORY__ + "/data/developer_survey_2017/survey_results_public.csv"
type SOData = CsvProvider<file>

In [ ]:
let stackoverflowSurvey = 
  let data = SOData.GetSample()
  data.Rows
  |> Seq.filter (fun row -> row.Country = "Poland" && not (Double.IsNaN(row.Salary))) 
  |> Seq.map (fun row -> row.Salary )
  |> Seq.toArray

In [ ]:
stackoverflowSurvey |> Array.length
stackoverflowSurvey |> Array.average
stackoverflowSurvey |> Array.max

In [ ]:
let soResults =
  mostLikelyDistribution histograms stackoverflowSurvey